In [ ]:
!pip install modelscope
!pip install transformers
!pip install streamlit
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install peft

# 下载模型

In [3]:
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2.5-1.5B-Instruct', cache_dir='/root/llms', revision='master')

2024-12-03 13:21:31,014 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


2024-12-03 13:22:50,111 - modelscope - INFO - Creating symbolic link [/root/llms/hub/qwen/Qwen2.5-1.5B-Instruct].
2024-12-03 13:22:50,112 - modelscope - WARNING - Failed to create symbolic link /root/llms/hub/qwen/Qwen2.5-1.5B-Instruct.


# 导入模型

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "/root/llms/qwen/Qwen2___5-1___5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
# 导入并适配数据集

### 导入数据集

In [17]:
from datasets import load_dataset

data_id="dataset/test2.json"
dataset = load_dataset("json", data_files=data_id)
print(dataset["train"])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1
})


In [15]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """
    prompt_text = ''                          # 所有数据前的指令文本
    max_input_length = 512                    # 输入的最大长度
    max_output_length = 1536                  # 输出的最大长度

    # 构建问题文本
    question = prompt_text + example['instruction']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['output']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

### 调整数据集

In [18]:
# 获取 'train' 部分的列名
column_names = dataset['train'].column_names

# 使用lambda函数调用tokenize_func函数，并传入example和tokenizer作为参数
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False,  # 不按批次处理
    remove_columns=column_names  # 移除特定列（column_names中指定的列）
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
    padding=True
)

# 配置LoRA

In [20]:
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['qwen2']

In [21]:
# 从peft库导入LoraConfig和get_peft_model函数
from peft import LoraConfig, get_peft_model, TaskType

# 创建一个LoraConfig对象，用于设置LoRA（Low-Rank Adaptation）的配置参数
config = LoraConfig(
    r=8,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=32,  # LoRA适应的比例因子
    # 指定需要训练的模型层的名字，不同模型对应层的名字不同
    # target_modules=["query_key_value"],
    target_modules=target_modules,
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
    # task_type="CAUSAL_LM"  # 任务类型，这里设置为因果(自回归）语言模型
    task_type=TaskType.CAUSAL_LM
)

# 使用get_peft_model函数和给定的配置来获取一个PEFT模型
model = get_peft_model(model, config)

# 打印出模型中可训练的参数
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


### 配置LoRA 超参数

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
        output_dir="./output/Qwen2.5_instruct_lora",  # 指定模型输出和保存的目录
        per_device_train_batch_size=4,  # 每个设备上的训练批量大小
        learning_rate=2e-4,  # 学习率
        fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
        logging_steps=20,  # 指定日志记录的步长，用于跟踪训练进度
        save_strategy="steps",   # 模型保存策略
        save_steps=50,   # 模型保存步数
        # max_steps=50, # 最大训练步长
        num_train_epochs=1  # 训练的总轮数
    )

### 开始训练

In [23]:
trainer = Trainer(
    model=model,  # 指定训练时使用的模型
    train_dataset=tokenized_dataset,  # 指定训练数据集
    args=training_args,
    data_collator=data_collator,
)

model.use_cache = False
# trainer.train()
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.1401021480560303, metrics={'train_runtime': 50.8608, 'train_samples_per_second': 0.02, 'train_steps_per_second': 0.02, 'total_flos': 2903506338816.0, 'train_loss': 1.1401021480560303, 'epoch': 1.0})

### 保存LoRA结果

In [ ]:
# lora_model_path = "lora/chatglm3-6b-int8"
# trainer.model.save_pretrained(lora_model_path )

# 测试LoRA结果

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/llms/qwen/Qwen2___5-1___5B-Instruct/'
lora_path = 'output/Qwen2.5_instruct_lora/checkpoint-1' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "#石狮融媒即时新闻# 【锦里村争分夺秒重建家园】7月31日上午，经过锦里村党支部组织的党员干部、网格员、村民代表、菲华青商会会员、志愿者们持续奋战，不仅消除了群众出行的安全隐患，而且村间道路也畅通了，锦里村的家园重拾美丽“容颜”。受超强台风“杜苏芮”影响，蚶江镇锦里村辖区出现树木倒伏、电杆倾倒、通行受阻等情况。台风过后锦里村迅速展开重建工作，大家齐心协力，一起恢复村主干道交通，清理道路两侧被大风折断的树枝、吹落的广告牌、钢结构等杂物。（石狮市融媒体中心记者 兰良增 报道）"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "接下来我会向你提供一些关于台风灾害的微博文本，你需要根据我提供的文本，分别解析出事件、省、市、区县与事件主题，且文本中可能含有多个事件及其对应位置。事件主题为人员受灾、住房与人居环境受灾、基础设施受灾、农业(含渔业)受灾、工业受灾、服务业受灾和洪涝，共计七类主题。结果以json格式返回。请注意，事件需要尽量保持完整，县级市需要归类为区县级，输出结果中省、市、区县不能为空值，必须具有推理结果。请注意，当发生覆盖区县为全市时，该解析结果不显示。请按照如下格式输出:```[{'事件': '', '省': '','市': '','区县': '','主题': '','详细地址': ''}]```"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

```json
[
    {
        "事件": "台风灾害",
        "省": "福建",
        "市": "泉州",
        "区县": "石狮市",
        "主题": "农业(含渔业)",
        "详细地址": "蚶江镇锦里村"
    }
]
```


### LoRA 连续对话测试

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os

# 可调参数，建议在文本生成时设置为较高值
TOP_P = 0.9        # Top-p (nucleus sampling)，范围0到1
TOP_K = 80         # Top-k 采样的K值
TEMPERATURE = 0.3  # 温度参数，控制生成文本的随机性

device = "cuda" if torch.cuda.is_available() else "cpu"

# 获取当前脚本目录，亦可改为绝对路径
mode_path = '/root/llms/qwen/Qwen2___5-1___5B-Instruct/'
lora_path = 'output/Qwen2.5_instruct_lora/checkpoint-1' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    mode_path,
    torch_dtype="auto",
    device_map="auto"
)
model = PeftModel.from_pretrained(model, model_id=lora_path)

tokenizer = AutoTokenizer.from_pretrained(mode_path)

# 系统指令（建议为空）
# messages = [
#     {"role": "system", "content": ""}
# ]
messages = [{"role": "user", "content": "接下来我会向你提供一些关于台风灾害的微博文本，你需要根据我提供的文本，分别解析出事件、省、市、区县与事件主题，且文本中可能含有多个事件及其对应位置。事件主题为人员受灾、住房与人居环境受灾、基础设施受灾、农业(含渔业)受灾、工业受灾、服务业受灾和洪涝，共计七类主题。结果以json格式返回。请注意，事件需要尽量保持完整，县级市需要归类为区县级，输出结果中省、市、区县不能为空值，必须具有推理结果。请注意，当发生覆盖区县为全市时，该解析结果不显示。请按照如下格式输出:```[{'事件': '', '省': '','市': '','区县': '','主题': '','详细地址': ''}]```"}]

while True:
    # 获取用户输入
    user_input = input("User: ").strip()

    # 添加用户输入到对话
    messages.append({"role": "user", "content": user_input})

    # 准备输入文本
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # 生成响应
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        top_p=TOP_P,
        top_k=TOP_K,
        temperature=TEMPERATURE,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id  # 避免警告
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # 解码并打印响应
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"Assistant: {response}")

    # 将生成的响应添加到对话中
    messages.append({"role": "assistant", "content": response})

User: #石狮融媒即时新闻# 【锦里村争分夺秒重建家园】7月31日上午，经过锦里村党支部组织的党员干部、网格员、村民代表、菲华青商会会员、志愿者们持续奋战，不仅消除了群众出行的安全隐患，而且村间道路也畅通了，锦里村的家园重拾美丽“容颜”。受超强台风“杜苏芮”影响，蚶江镇锦里村辖区出现树木倒伏、电杆倾倒、通行受阻等情况。台风过后锦里村迅速展开重建工作，大家齐心协力，一起恢复村主干道交通，清理道路两侧被大风折断的树枝、吹落的广告牌、钢结构等杂物。（石狮市融媒体中心记者 兰良增 报道）


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Assistant: ```json
[
    {
        "事件": "台风导致道路受损",
        "省": "",
        "市": "石狮市",
        "区县": "蚶江镇锦里村",
        "主题": "洪涝",
        "详细地址": "蚶江镇锦里村"
    }
]
```
User: #头条新闻# 【#北京暴雨四年级小学生街头清扫积水#：我是小区的一员】7月30日，#北京暴雨# 。朝阳区化工路垡头段出现严重积水，垡头街道组织各方力量排除积水隐患。记者在现场发现了一名四年级的小学生，拿着一把扫帚，在现场努力推动着积水。他说：我是小区一员，不能让这个小区里都是水。#受台风影响你明天正常上班吗#  http://t.cn/A609aZew
Assistant: ```json
[
    {
        "事件": "台风引发积水",
        "省": "",
        "市": "北京市",
        "区县": "朝阳区垡头街道",
        "主题": "洪涝",
        "详细地址": "垡头街道化工路垡头段"
    },
    {
        "事件": "小学生参与社区服务",
        "省": "",
        "市": "北京市",
        "区县": "朝阳区垡头街道",
        "主题": "洪涝",
        "详细地址": "垡头街道化工路垡头段"
    }
]
```


KeyboardInterrupt: Interrupted by user

# 合并LoRA结果

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path="/root/work/chatglm3-6b"
peft_model_path="./lora/chatglm3-6b-int8"
save_path = "chatglm3-6b-lora"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_path)
model = model.merge_and_unload()

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

2024-12-01 23:59:36.495648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 23:59:37.718081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OSError: Incorrect path_or_model_id: '/root/work/chatglm3-6b'. Please provide either the path to a local folder or the repo_id of a model on the Hub.